In [ ]:
# bzq modifying
len_x_source = 3
len_y_source = 3
stride_x_source = 2
stride_y_source = 2

len_x_target = 5
len_y_target = 5
stride_x_target = 1
stride_y_target = 1

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 60
pixels = 0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os

# Define the ResNet-20 V1 architecture
def resnet_v1(input_shape, num_classes):
    def resnet_block(inputs, filters, kernel_size=3, stride=1, activation='relu'):
        x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        return x

    inputs = layers.Input(shape=input_shape)
    x = resnet_block(inputs, 16)
    for _ in range(3):
        x = resnet_block(x, 16)
    for _ in range(3):
        x = resnet_block(x, 32, stride=2)
    for _ in range(3):
        x = resnet_block(x, 64, stride=2)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

# Define model
model = resnet_v1(input_shape=(32, 32, 3), num_classes=10)

# Compile model
initial_lr = 0.001
model.compile(optimizer=optimizers.Adam(learning_rate=initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate schedule
def lr_schedule(epoch):
    lr = initial_lr
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    return lr

lr_scheduler = callbacks.LearningRateScheduler(lr_schedule)

# Train model
batch_size = 128
epochs = 200
if os.path.exists('resnet.weights.h5'):
    model.load_weights("resnet.weights.h5")
else:
    model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
            validation_data=(x_test, y_test),
            epochs=epochs,
            callbacks=[lr_scheduler])

# Save weights for LeNet model
model.save_weights('resnet.weights.h5')

In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle


len_x = len_x_source
len_y = len_y_source
stride_x = stride_x_source
stride_y = stride_y_source

bzq = []
#bzq_acc = []
#bzq_conf = []

correct_predictions = []
incorrect_predictions = []

def single_data_bzq_mask_preprocessing(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[start_y:start_y + len_y, start_x:start_x + len_x, 0] *= magnification
    new_data[start_y:start_y + len_y, start_x:start_x + len_x, 1] *= magnification
    new_data[start_y:start_y + len_y, start_x:start_x + len_x, 2] *= magnification
    return new_data

# Ensure x_test and y_test are defined
x_test = np.array(x_test)
y_test = np.array(y_test)

original_predictions = model.predict(x_test, verbose=0)

for i in range(len(original_predictions)):
    if np.argmax(original_predictions[i]) == np.argmax(y_test[i]):
        correct_predictions.append(i)
    else:
        incorrect_predictions.append(i)

# 從檔案讀取變數
if os.path.exists('variables.pkl'):
    print("exists")
    with open('variables.pkl', 'rb') as f:
        bzq, correct_predictions, incorrect_predictions = pickle.load(f)
else:
    for k in range(len(x_test)):
        single_data_bzq_classification_record = []
        targets = []
        for i in range(0, 32 - len_y, stride_y):
            for j in range(0, 32 - len_x, stride_x):
                target = single_data_bzq_mask_preprocessing(x_test[k], i, j, len_x, len_y, 0)
                targets.append(target)

        targets = np.array(targets)

        # Ensure all targets have the same shape
        targets = np.array([np.expand_dims(t, axis=0) if t.ndim == 3 else t for t in targets])

        # 批次預測
        predictions = model.predict(np.vstack(targets), verbose=0)
        max_bzq_indices = np.argmax(predictions, axis=1)
        
        single_data_bzq_classification_record.extend(max_bzq_indices)
        
        counter = Counter(single_data_bzq_classification_record)
        most_common_num, most_common_count = counter.most_common(1)[0]
        
        bzq.append((len(single_data_bzq_classification_record) - most_common_count) / len(single_data_bzq_classification_record))
        #bzq_acc.append(np.sum(np.array(single_data_bzq_classification_record) == np.argmax(y_test[k])) / len(single_data_bzq_classification_record))
        #bzq_conf.append(most_common_count / len(single_data_bzq_classification_record))

        original_data = single_data_bzq_mask_preprocessing(x_test[k], 0, 0, 0, 0, 0)
        
        # Ensure original_data has the correct shape
        if original_data.ndim == 3:
            original_data = np.expand_dims(original_data, axis=0)
        
        original_prediction = model.predict(original_data, verbose=0)

        max_original_index = np.argmax(original_prediction)
        # 將變數保存到檔案
        with open('variables.pkl', 'wb') as f:
            pickle.dump((bzq, correct_predictions, incorrect_predictions), f)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# 假設需要的library都有import進去且所有陣列都存在
bzq_correct = np.array([bzq[i] for i in correct_predictions])
bzq_incorrect = np.array([bzq[i] for i in incorrect_predictions])

x_bzq_correct = []
y_bzq_correct = []
x_bzq_incorrect = []
y_bzq_incorrect = []


print(len(bzq_correct))
print(len(bzq_incorrect))

# 繪製正確預測的直方圖
x_bzq_correct_stat = []
y_bzq_correct_stat = []

plt.figure(figsize=(12, 8))
counts, bins, patches = plt.hist(bzq_correct, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置

# 在每個柱子上標示數字，排除數值為 0 的情況
for count, bin in zip(counts, bins[:-1]):
    if count > 0:
        #plt.text(bin + (bins[1] - bins[0]) / 2, count, f'({bin:.3f}, {int(count)})', ha='center', va='bottom')
        x_bzq_correct_stat.append(bin + (bins[1] - bins[0]) / 2)
        y_bzq_correct_stat.append(count)

plt.show()

# 顯示整理後的數值成表格
data = {'bzq': x_bzq_correct_stat, 'Count': y_bzq_correct_stat}
df = pd.DataFrame(data)

# 繪製表格圖片
fig, ax = plt.subplots(figsize=(6, 6))  # 設置圖片大小
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='left', loc='left')

plt.show()

# 繪製錯誤預測的直方圖
x_bzq_incorrect_stat = []
y_bzq_incorrect_stat = []

plt.figure(figsize=(12, 8))
counts, bins, patches = plt.hist(bzq_incorrect, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置

# 在每個柱子上標示數字，排除數值為 0 的情況
for count, bin in zip(counts, bins[:-1]):
    if count > 0:
        #plt.text(bin + (bins[1] - bins[0]) / 2, count, f'({bin:.3f}, {int(count)})', ha='center', va='bottom')
        x_bzq_incorrect_stat.append(bin + (bins[1] - bins[0]) / 2)
        y_bzq_incorrect_stat.append(count)

plt.show()

# 顯示整理後的數值成表格
data = {'bzq': x_bzq_incorrect_stat, 'Count': y_bzq_incorrect_stat}
df = pd.DataFrame(data)

# 繪製表格圖片
fig, ax = plt.subplots(figsize=(6, 6))  # 設置圖片大小
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='left', loc='left')

plt.show()

# 繪製四分位線圖
plt.figure(figsize=(10, 6))
box = plt.boxplot([bzq_correct, bzq_incorrect], labels=['bzq_correct', 'bzq_incorrect'], patch_artist=True)

# 計算並標示每個重要節點
data = [bzq_correct, bzq_incorrect]
for i, dataset in enumerate(data):
    q1 = pd.Series(dataset).quantile(0.25)
    median = pd.Series(dataset).median()
    q3 = pd.Series(dataset).quantile(0.75)
    min_val = min(dataset)
    max_val = max(dataset)
    
    plt.text(i + 1.1, q1, f'Q1: {q1:.3f}', horizontalalignment='left', verticalalignment='center', fontsize=10, color='blue')
    plt.text(i + 1.1, median, f'Median: {median:.3f}', horizontalalignment='left', verticalalignment='center', fontsize=10, color='green')
    plt.text(i + 1.1, q3, f'Q3: {q3:.3f}', horizontalalignment='left', verticalalignment='center', fontsize=10, color='blue')
    plt.text(i + 1.1, min_val, f'Min: {min_val:.3f}', horizontalalignment='left', verticalalignment='center', fontsize=10, color='red')
    plt.text(i + 1.1, max_val, f'Max: {max_val:.3f}', horizontalalignment='left', verticalalignment='center', fontsize=10, color='red')

plt.title('Box plot')
plt.ylabel('bzq')
plt.show()

# 對比
plt.hist(bzq_correct, bins=bins_size, alpha=0.5, label='Correct')
plt.hist(bzq_incorrect, bins=bins_size, alpha=0.5, label='Incorrect')

plt.title('Histogram of Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 添加圖例以區分兩組數據
plt.show()

x_bzq_correct = np.array(x_bzq_correct)
y_bzq_correct = np.array(y_bzq_correct)
threshold_bzq_correct = pd.Series(bzq_correct).quantile(0.75) + 1.5 * (pd.Series(bzq_correct).quantile(0.75) - pd.Series(bzq_correct).quantile(0.25))
print(f"threshold_bzq_correct: {threshold_bzq_correct}")

'''# 過濾異常值(Q3 + 1.5IRQ)
for i in range(len(x_bzq_correct_stat)):
    if x_bzq_correct_stat[i] > threshold_bzq_correct:
        y_bzq_correct_stat[i] = 0'''

x_bzq_incorrect = np.array(x_bzq_incorrect)
y_bzq_incorrect = np.array(y_bzq_incorrect)


# 顯示整理後的數值成表格
data = {'bzq': x_bzq_correct_stat, 'Count': y_bzq_correct_stat}
df = pd.DataFrame(data)

# 繪製表格圖片
fig, ax = plt.subplots(figsize=(6, 6))  # 設置圖片大小
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='left', loc='left')

plt.show()


In [ ]:
# 將正確的做成回歸得到f(x)、錯誤的做回歸得到g(x)，再 f / (f + g)

from sklearn.metrics import r2_score
from scipy.interpolate import interp1d

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


x_bzq_correct_stat = moving_average(np.array(x_bzq_correct_stat), window_size)
y_bzq_correct_stat = moving_average(np.array(y_bzq_correct_stat), window_size)

# 將數據轉換為二維陣列
X = x_bzq_correct_stat[:, np.newaxis]

# 創建多項式特徵
poly_bzq_correct = PolynomialFeatures(degree=poly_degree)
X_poly_bzq_correct = poly_bzq_correct.fit_transform(X)

# 創建線性回歸模型並擬合數據
model_lr_bzq_correct = LinearRegression()
model_lr_bzq_correct.fit(X_poly_bzq_correct, y_bzq_correct_stat)

# 預測數據
y_poly_pred_bzq_correct = model_lr_bzq_correct.predict(X_poly_bzq_correct)

# 獲取模型的係數和截距
coefficients_bzq_correct = model_lr_bzq_correct.coef_
intercept_bzq_correct = model_lr_bzq_correct.intercept_

# 計算 R² 分數
r2_bzq_correct = r2_score(y_bzq_correct_stat, y_poly_pred_bzq_correct)

print(coefficients_bzq_correct)
print(intercept_bzq_correct)
print(r2_bzq_correct)

# 繪製數據點和回歸曲線
plt.scatter(x_bzq_correct_stat, y_bzq_correct_stat, color='blue', label='Data Points')
plt.plot(x_bzq_correct_stat, y_poly_pred_bzq_correct, color='red', label='Cubic Regression')
plt.xlabel('x_bzq_correct_stat')
plt.ylabel('y_bzq_correct_stat')
plt.legend()
plt.show()


x_bzq_incorrect_stat = moving_average(np.array(x_bzq_incorrect_stat), window_size)
y_bzq_incorrect_stat = moving_average(np.array(y_bzq_incorrect_stat), window_size)
# 將數據轉換為二維陣列
X = x_bzq_incorrect_stat[:, np.newaxis]

# 創建多項式特徵
poly_bzq_incorrect = PolynomialFeatures(degree=poly_degree)
X_poly_bzq_incorrect = poly_bzq_incorrect.fit_transform(X)

# 創建線性回歸模型並擬合數據
model_lr_bzq_incorrect = LinearRegression()
model_lr_bzq_incorrect.fit(X_poly_bzq_incorrect, y_bzq_incorrect_stat)

# 預測數據
y_poly_pred_bzq_incorrect = model_lr_bzq_incorrect.predict(X_poly_bzq_incorrect)

# 獲取模型的係數和截距
coefficients_bzq_incorrect = model_lr_bzq_incorrect.coef_
intercept_bzq_incorrect = model_lr_bzq_incorrect.intercept_

# 計算 R² 分數
r2_bzq_incorrect = r2_score(y_bzq_incorrect_stat, y_poly_pred_bzq_incorrect)

print(coefficients_bzq_incorrect)
print(intercept_bzq_incorrect)
print(r2_bzq_incorrect)

# 繪製數據點和回歸曲線
plt.scatter(x_bzq_incorrect_stat, y_bzq_incorrect_stat, color='blue', label='Data Points')
plt.plot(x_bzq_incorrect_stat, y_poly_pred_bzq_incorrect, color='red', label='Cubic Regression')
plt.xlabel('x_bzq_correct_stat')
plt.ylabel('y_bzq_correct_stat')
plt.legend()
plt.show()

# 構建 f(x) 和 g(x)
def polynomial_function(coefficients, intercept, X):
    return np.polyval(coefficients[::-1], X) + intercept

# 使用相同的 x 值來計算 f(x) 和 g(x)
x_values = np.linspace(min(x_bzq_correct_stat), max(x_bzq_correct_stat), 10000)
f_x = np.maximum(0, polynomial_function(coefficients_bzq_correct, intercept_bzq_correct, x_values))
g_x = np.maximum(0, polynomial_function(coefficients_bzq_incorrect, intercept_bzq_incorrect, x_values))

# 計算 f(x) / (f(x) + g(x))
result = f_x / (f_x + g_x)

# 打印結果
print("f(x):", f_x)
print("g(x):", g_x)
print("f(x) / (f(x) + g(x)):", result)

# 繪製 f(x), g(x) 的圖形
plt.plot(x_values, f_x, label='f(x)')
plt.plot(x_values, g_x, label='g(x)')
plt.xlabel('x_bzq_stat')
plt.ylabel('y_bzq_stat')
plt.legend()
plt.show()

# 繪製 f(x) / (f(x) + g(x)) 的圖形
plt.plot(x_values, result, color='purple', label='f(x) / (f(x) + g(x))')
plt.xlabel('x_bzq_stat')
plt.ylabel('f(x) / (f(x) + g(x))')
plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
# 劃出曲線，預測mnist

#原始資料集的training data用沒有 mask 的模型做訓練得到weights A
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Define LeNet model
def create_lenet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    return model

# Define MLP model
def create_mlp():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    return model

# Compile and train model
def train_model(model, x_train, y_train, x_val, y_val, epochs=20, batch_size=128):
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[early_stopping])

# Split data for validation
x_val, y_val = x_train[-10000:], y_train[-10000:]
x_train, y_train = x_train[:-10000], y_train[:-10000]

# Train LeNet model
lenet_model = create_lenet()
if os.path.exists("lenet.weights.h5"):
    lenet_model.load_weights("lenet.weights.h5")
else:
    train_model(lenet_model, x_train, y_train, x_val, y_val)

# Train MLP model
#mlp_model = create_mlp()
#train_model(mlp_model, x_train, y_train, x_val, y_val)

# Ensemble predictions
def ensemble_predictions(models, x_test):
    predictions = [model.predict(x_test) for model in models]
    return np.mean(predictions, axis=0)

# Create ensemble of LeNet models
#lenet_ensemble = [create_lenet() for _ in range(10)]
#for model in lenet_ensemble:
#    train_model(model, x_train, y_train, x_val, y_val)

# Get ensemble predictions
#ensemble_preds = ensemble_predictions(lenet_ensemble, x_test)

# Save weights for LeNet model
lenet_model.save_weights('lenet.weights.h5')

# Save weights for MLP model
#mlp_model.save_weights('mlp.weights.h5')

# Save weights for each model in the ensemble
#for i, model in enumerate(lenet_ensemble):
#    model.save_weights(f'lenet_ensemble.weights_{i}.h5')



In [ ]:
# 得到了bzq 正確的函數，拿來做mnist 正確的預測
import numpy as np
import tensorflow as tf
from collections import Counter
import scipy.ndimage

def single_data_rotate_roll_preprocessing(original_data, angle, pixels):
    new_data = np.copy(original_data).astype(np.float32)
    #new_data = np.expand_dims(new_data, axis=-1)
    new_data = scipy.ndimage.rotate(new_data, angle, reshape=False)
    new_data = np.expand_dims(new_data, axis=0)  # 增加批次維度
    new_data = np.roll(new_data, pixels, axis=-1)
    return new_data

def single_data_bzq_mask_preprocessing_mnist(original_data, start_x, start_y, len_x, len_y, magnification):
    new_data = np.copy(original_data)
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    new_data[start_y:start_y + len_y, start_x:start_x + len_x] = 1
    return new_data

lenet_model = create_lenet()
lenet_model.load_weights('lenet.weights.h5')

bzq = []

correct_predictions_mnist = []
incorrect_predictions_mnist = []

x_test = np.array(x_test)

# 預先計算旋轉和滾動的資料
preprocessed_data = np.array([single_data_rotate_roll_preprocessing(x_test[k], angle, pixels) for k in range(10000)]).reshape(-1, 28, 28, 1)

# 顯示影像
for i in range(2):
    plt.imshow(preprocessed_data[i, :, :, 0], cmap='gray')
plt.show()

original_predictions_mnist = lenet_model.predict(preprocessed_data, verbose=0)
for i in range(10000):
    if np.argmax(original_predictions_mnist[i]) == np.argmax(y_test[i]):
        correct_predictions_mnist.append(i)
    else:
        incorrect_predictions_mnist.append(i)

# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target

for k in range(len(x_test)):
    single_data_bzq_classification_record = []
    targets = []
    for i in range(0, 28 - len_y, stride_y):
        for j in range(0, 28 - len_x, stride_x):
            target = single_data_bzq_mask_preprocessing_mnist(preprocessed_data[k], i, j, len_x, len_y, 0)
            targets.append(target)

    # 批次預測
    predictions = lenet_model.predict(np.vstack(targets).reshape(-1, 28, 28, 1), verbose=0)
    max_bzq_indices = np.argmax(predictions, axis=1)
    
    single_data_bzq_classification_record.extend(max_bzq_indices)
    
    counter = Counter(single_data_bzq_classification_record)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    bzq.append((len(single_data_bzq_classification_record) - most_common_count) / len(single_data_bzq_classification_record))
    #bzq_acc.append(np.sum(single_data_bzq_classification_record == np.argmax(y_test[k])) / len(single_data_bzq_classification_record))
    #bzq_conf.append(most_common_count / len(single_data_bzq_classification_record))

    original_data = single_data_bzq_mask_preprocessing_mnist(preprocessed_data[k], 0, 0, 0, 0, 0)
    original_prediction = lenet_model.predict(original_data.reshape(-1, 28, 28, 1), verbose=0)

    max_original_index = np.argmax(original_prediction)



In [ ]:
print(len(preprocessed_data))
print(f"{len(correct_predictions_mnist)}, {len(incorrect_predictions_mnist)}")
#print(bzq)
bzq_mnist = np.array(bzq)

f_bzq_mnist = np.polyval(result, bzq)

f_bzq_mnist = np.maximum(0, polynomial_function(coefficients_bzq_correct, intercept_bzq_correct, bzq_mnist))
g_bzq_mnist = np.maximum(0, polynomial_function(coefficients_bzq_incorrect, intercept_bzq_incorrect, bzq_mnist))

result_bzq_mnist = f_bzq_mnist / (f_bzq_mnist + g_bzq_mnist)
for i, val in enumerate(result_bzq_mnist):
    if val > 1:
        result_bzq_mnist[i] = 1
    elif val < 0:
        result_bzq_mnist[i] = 0
        
#print(result_bzq_mnist)

counts, bins, patches = plt.hist(bzq_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()

# 打印結果
plt.boxplot(bzq_mnist)
plt.show()

# 繪製點狀圖
plt.scatter(bzq, result_bzq_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f/f+g')
plt.xlabel('x')
plt.ylabel('f/f+g')

# 顯示圖表
plt.show()

#待確認
'''bzq_correct_mnist = np.array([bzq_mnist[i] for i in correct_predictions])
bzq_incorrect_mnist = np.array([bzq_mnist[i] for i in incorrect_predictions])

result_bzq_correct_mnist = np.array([result_bzq_mnist[i] for i in correct_predictions])
result_bzq_incorrect_mnist = np.array([result_bzq_mnist[i] for i in incorrect_predictions])'''

bzq_correct_mnist = np.array([bzq_mnist[i] for i in correct_predictions_mnist])
bzq_incorrect_mnist = np.array([bzq_mnist[i] for i in incorrect_predictions_mnist])

result_bzq_correct_mnist = np.array([result_bzq_mnist[i] for i in correct_predictions_mnist])
result_bzq_incorrect_mnist = np.array([result_bzq_mnist[i] for i in incorrect_predictions_mnist])

# 打印結果

counts, bins, patches = plt.hist(bzq_correct_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()
plt.boxplot(bzq_correct_mnist)
plt.show()
# 繪製點狀圖
plt.scatter(bzq_correct_mnist, result_bzq_correct_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f/f+g')
plt.xlabel('x')
plt.ylabel('f/f+g')

# 顯示圖表
plt.show()

# 打印結果
counts, bins, patches = plt.hist(bzq_incorrect_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()
plt.boxplot(bzq_incorrect_mnist)
plt.show()
# 繪製點狀圖
plt.scatter(bzq_incorrect_mnist, result_bzq_incorrect_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f/f+g')
plt.xlabel('x')
plt.ylabel('f/f+g')

# 顯示圖表
plt.show()

In [ ]:
# 劃出confidence-acc 圖: confidence由bzq提供，acc由該confidence數值底下預測準確的

result_pred_mnist = np.ones(10000) 
for i in incorrect_predictions_mnist:
    result_pred_mnist[i] = 0

print(sum(result_pred_mnist))

result_mnist_dict = {}
for i, val in enumerate(result_bzq_mnist):
    if val not in result_mnist_dict.keys():
        result_mnist_dict[val] = [result_pred_mnist[i]]
    else:
        result_mnist_dict[val].append(result_pred_mnist[i])

# 根據key進行排序
#result_mnist_dict = dict(sorted(result_mnist_dict.items(), reverse=True))

#print(result_mnist_dict)

# 初始化信心值和準確率列表
confidence_values = []
accuracies = []
element_counts = []

# 計算每個信心值範圍的準確率
for confidence in sorted(result_mnist_dict.keys(), reverse=True):
    combined_results = []
    for key in result_mnist_dict:
        if key >= confidence:
            combined_results.extend(result_mnist_dict[key])
    element_count = len(combined_results)
    accuracy = np.mean(combined_results)
    confidence_values.append(confidence)
    accuracies.append(accuracy)
    element_counts.append(element_count)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, accuracies, marker='o', linestyle=' ', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, element_counts, marker='o', linestyle=' ', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Number of Elements (p(y|x) >= τ)')
plt.title('Confidence Threshold vs Number of Elements')
plt.grid(True)
plt.show()

In [ ]:
print(sorted(result_mnist_dict, reverse=True))